In [1]:
import boto3
import json
import time
from dotenv import load_dotenv
from datetime import datetime
import os

# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [2]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DeltaTableReader") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0,org.apache.hadoop:hadoop-aws:3.3.1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Path to the Delta table in S3
delta_table_path = "s3a://orders-for-dispatch/ready_for_dispatch"


24/09/30 20:44:35 WARN Utils: Your hostname, Somnium.local resolves to a loopback address: 127.0.0.1; using 172.28.59.194 instead (on interface en0)
24/09/30 20:44:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/borja/.ivy2/cache
The jars for the packages stored in: /Users/borja/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-94836227-bc3e-4d36-83c8-692d4cf8b10c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.1!hadoop-aws.jar (97ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.901/aws-java-sdk-bundle-1.11.901.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-

In [4]:
# Path to the Delta table in S3
delta_table_path = 's3a://orders-for-dispatch/ready_for_dispatching'

In [7]:
from delta.tables import DeltaTable

# Function to read records marked as ready for dispatch
def read_ready_for_dispatch_orders():
    # Load the Delta table
    delta_table = DeltaTable.forPath(spark, delta_table_path)
    
    # Query the Delta table to fetch only the records marked as ready for dispatch
    df = delta_table.toDF().filter("status = 'ready_for_dispatch'")
    
    # Show the records
    df.show(truncate=False)

    # You can return the DataFrame or process it further
    return df

# Read the ready-for-dispatch orders
ready_orders_df = read_ready_for_dispatch_orders()

+------------+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------+------------------------------------+------------------+-------------------+
|client_id   |location                                                                                                             |order                                           |order_id                            |status            |timestamp          |
+------------+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------+------------------------------------+------------------+-------------------+
|966388710620|{lon -> -3.691248605006199, address -> Pasadizo de Elvira Murcia, 22, Madrid, Spain, lat -> 40.52362535729707}       |{volume -> null, n_objects -> 8, weight -> null}|a764f612-d877-477e-8798-ee45543e950f|ready_fo